# Sheet 04

## Preamble

Autors: Marten Ringwelski, Nico Ostermann, Simon Liessem

Note that this notebook MUST be executed in order to get everything to work.
The tasks can't be run individually. 

Also eCampus does not allow for uploading nested directory structures which makes it hard to properly organize the files. The files are expected to be in the `data` directory which itself is placed next to this notebook.

If you extract the zip file we handed in everything should work just fine.

Autoformatting if `jupyter-black` is installed.

In [ ]:
try:
    import black
    import jupyter_black

    jupyter_black.load(
        lab=False,
        line_length=79,
        verbosity="DEBUG",
        target_version=black.TargetVersion.PY310,
    )
except ImportError:
    pass

Import all we weed and more.

Set seaborn default theme

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.feature_selection import f_classif, SelectKBest
import math as m
import plotly.express as px
import sklearn.manifold
import sklearn.discriminant_analysis
import scipy as sp
import scipy.sparse

Set seaborn default theme

In [ ]:
sns.set_theme()

If needed tweak parameters of matplotlib.
Here we increase the size and dpi to bet a bigger but still high-res image.

In [ ]:
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["figure.figsize"] = (20, 15)
%matplotlib inline

Disable future warnings as we get a lot of them and don't really care for this sheet.

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

## Exercise 1

### a)

Read the dataframe and replace missing values by the respective mean of the column.

In [ ]:
df = pd.read_excel("data/breast-cancer-wisconsin.xlsx")
df = df.fillna(df.mean())

df["class"] = df["class"].map({2: "benign", 4: "malignant"})

Now define the DataFrame for t-SNE and create a new one with the result.

In [ ]:
data_columns = df.columns.difference(["class", "code"])

In [ ]:
df_wo_meta = df[data_columns]

Do t-SNE with different perplexities as the task asked us to.


In [ ]:
perplexities = [5, 10, 20, 30, 40, 50]
fig, axs = plt.subplots(
    nrows=2,
    ncols=m.ceil(len(perplexities) / 2),
)
for perplexity, ax in zip(perplexities, axs.flatten()):
    tsne = sk.manifold.TSNE(
        n_components=2,
        perplexity=perplexity,
        init="random",
        learning_rate="auto",
    )

    df_tsne = pd.DataFrame(
        tsne.fit_transform(df_wo_meta),
        columns=["x-tsne", "y-tsne"],
        index=df.index,
    )
    df_tsne[df.columns] = df

    ax.set_title(f"Perplexity: {perplexity}")
    ax.set_aspect("equal")
    sns.scatterplot(
        data=df_tsne,
        x="x-tsne",
        y="y-tsne",
        hue="class",
        ax=ax,
    )

Copy paste from above but init="pca"

In [ ]:
perplexities = [5, 10, 20, 30, 40, 50]
fig, axs = plt.subplots(
    nrows=2,
    ncols=m.ceil(len(perplexities) / 2),
)
for perplexity, ax in zip(perplexities, axs.flatten()):
    tsne = sk.manifold.TSNE(
        n_components=2,
        perplexity=perplexity,
        init="pca",
        learning_rate="auto",
    )

    df_tsne = pd.DataFrame(
        tsne.fit_transform(df_wo_meta),
        columns=["x-tsne", "y-tsne"],
        index=df.index,
    )
    df_tsne[df.columns] = df

    ax.set_title(f"Perplexity: {perplexity}")
    ax.set_aspect("equal")
    sns.scatterplot(
        data=df_tsne,
        x="x-tsne",
        y="y-tsne",
        hue="class",
        ax=ax,
    )

We see that with a higher perplexity the classes become linearly separable.
Also the classes become more dense.

We see a slight difference between random and pca initialisation.
With PCA initialisation and a perplexity of 10 the classes are slightly more dense than with random initialisation.
It makes sense that PCA initialisation is slightly better because they contain more information about the real data than random points.

Overall for perplexities greater euqal 20 the classes are nicely separated.

### b)

Read data and use mean for missing data.

In [ ]:
df = pd.read_excel(
    "data/Data_Cortex_Nuclear.xls",
    index_col="MouseID",
)
df = df.fillna(df.mean())

In [ ]:
meta_columns = ["Genotype", "Treatment", "Behavior", "class"]
df_wo_meta = df[df.columns.difference(meta_columns)]

df_scs = df_wo_meta[
    np.logical_or(
        df["class"] == "c-SC-s",
        df["class"] == "t-SC-s",
    )
].copy()

Now actuall do PCA and create a DataFrame with the result.
Also we use equal axis scale for the plot which makes sense since we care about the results from PCA.

In [ ]:
pca = sk.decomposition.PCA(
    n_components=2,
)
# XXX There must be a better way to do this
df_pca = pd.DataFrame(
    pca.fit_transform(df_scs),
    columns=["x-pca", "y-pca"],
    index=df_scs.index,
)
df_pca[df.columns] = df

In [ ]:
plt.gca().set_aspect("equal")

sns.scatterplot(
    df_pca,
    x="x-pca",
    y="y-pca",
    hue="class",
)

Now we do the same thing but for isomap.
As we see by the warnings using 2 or 5 is not good as the resulting graph has more than one connecting component.

In [ ]:
# We don't care that the calculation is expensive
warnings.simplefilter(
    action="ignore",
    category=sp.sparse.SparseEfficiencyWarning,
)
n_neighbors_array = [2, 5, 7, 13, 17, 23, 29]

fig, axs = plt.subplots(
    nrows=2,
    ncols=m.ceil(len(n_neighbors_array) / 2),
)

for n_neighbors, ax in zip(n_neighbors_array, axs.flatten()):
    isomap = sk.manifold.Isomap(
        n_neighbors=n_neighbors,
    )

    df_isomap = pd.DataFrame(
        isomap.fit_transform(df_scs),
        columns=["x-isomap", "y-isomap"],
        index=df_scs.index,
    )

    df_isomap[df.columns] = df

    plt.gca().set_aspect("equal")

    ax.set_title(f"n_neighbors: {n_neighbors}")
    sns.scatterplot(
        df_isomap,
        x="x-isomap",
        y="y-isomap",
        hue="class",
        ax=ax,
    )

We see that for n_neighbors >=5 the data is nicely separated but one could argue that you can see more than two classes.
For n_neighbors >=13 they are nicely separeted in just two classes.

Now we do t-SNE.

In [ ]:
perplexities = [2, 5, 7, 13, 17, 23, 29, 51]

fig, axs = plt.subplots(
    nrows=2,
    ncols=m.ceil(len(perplexities) / 2),
)

for perplexity, ax in zip(perplexities, axs.flatten()):
    tsne = sk.manifold.TSNE(
        perplexity=perplexity,
    )

    df_tsne = pd.DataFrame(
        tsne.fit_transform(df_scs),
        columns=["x-tsne", "y-tsne"],
        index=df_scs.index,
    )

    df_tsne[df.columns] = df

    plt.gca().set_aspect("equal")

    ax.set_title(f"perplexity: {perplexity}")
    sns.scatterplot(
        df_tsne,
        x="x-tsne",
        y="y-tsne",
        hue="class",
        ax=ax,
    )

We adjusted the perplexity as (according to the scipy docs) it is the only parameter relevant for our purpose.

For perplexity >=13 we get similar results to Isomap with n_neighbors>=7.

## Exercise 2

### a)

We note that an individual sum element of the sum that defines $S_c$ is a matrix whose columns are all a scaled version of $x_i - x_{mean}$.
So overall we need to sum over p Matricies to get p linearly independent columns.
This means that we need at least p points.

### b)

In [ ]:
df = pd.read_csv("data/LDA-input.csv")
lda = sk.discriminant_analysis.LinearDiscriminantAnalysis(solver="eigen")

In [ ]:
lda.fit(
    df.drop(columns="class"),
    df["class"],
)

In [ ]:
transformed_df = pd.DataFrame(
    lda.transform(df.drop(columns="class")),
    columns=["x", "y"],
)
transformed_df["class"] = df["class"]

In [ ]:
sns.scatterplot(
    data=transformed_df,
    x="x",
    y="y",
    hue="class",
)

### c)

In [ ]:
lda = sk.discriminant_analysis.LinearDiscriminantAnalysis(
    solver="eigen",
    shrinkage="auto",
)

In [ ]:
lda.fit(df.drop(columns="class"), df["class"])

In [ ]:
transformed_df = pd.DataFrame(
    lda.transform(df.drop(columns="class")),
    columns=["x", "y"],
)
transformed_df["class"] = df["class"]

In [ ]:
sns.scatterplot(
    data=transformed_df,
    x="x",
    y="y",
    hue="class",
)

We prefer this plot over the other plot.
In the other plot the x-Axis has a huge range so we cannot see differences in the x dimension in a class.
This plot has the x axis scaled in another way so we can see even relativly small changes.

### d)

We observe that the x-axis has a lower range and thus small differences in the x dimension can better be seen.

# Exercise 3

### a)

If the perplexity is the same as the amount of points, the neighborhood scale is set so large that all probablilties are approximatly equal.
A large neighborhood scale means a flattened out gauss curve.

If all probabilities are equal the KL Divergence is minimized by choosing the distribution Q as uniform distribution for each point y.
So the result is as rather a uniformly distributed set of points.

If the perplexity is 29 all neighborhood probabilities but one are approximatly equal.
The different one is significantly lower than the others.
Due to this fact after a lot of steps the classes properly separate.

### b)

The points at the corners pull their close neighbors strongly and the farther away points weakly.
This leads to the small distances.

The points in the middle are pulled in all directions euqally strong which results to them staying in the middle.

### c)

We do observe that it breaks down into seperate clusters.
Since the perplexity is 2 we have a rather high chance that some points distance themselves from the big cluster and form smaller clusters.
This happens because if a single point insolates itself, other neighbouring points have a chance to choose the isolated point instead of the bigger cluster. 

### d)

The dataset is a dense circle.
With low perplexity points that are actually close together are not seen as close together by tSNE.
So a high perplexity is needed to actually connect enough points to recreate the shape.

# Exercise 4

In [ ]:
from collections import namedtuple
from pprint import pformat
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt

In [ ]:
class Node(namedtuple("Node", "location left_child right_child")):
    def __repr__(self):
        return pformat(self._asdict())

In [ ]:
def kdtree(points, axis=0):
    """Given
     - a 2D numpy array points, where each column denotes a dimension and each
       row a datapoint
     - an integer axis indicating the dimension to split at the top level
    this recursive function creates and returns a KDTree as it was discussed
    in the lecture.
    """
    if points.shape[0] == 0:
        return None

    k = points.shape[1]  # assumes all points have the same dimension

    # Sort point list by axis and choose median as pivot element
    points = points[points[:, axis].argsort()]
    median = points.shape[0] // 2

    # Create node and construct subtrees
    return Node(
        location=points[median],
        left_child=kdtree(points[:median], (axis + 1) % k),
        right_child=kdtree(points[median + 1 :], (axis + 1) % k),
    )

### a)

In [ ]:
def one_NN_rec(tree: Node, query, neighbor=None, axis: int = 0):
    """
    This recursive function accepts
     - a KDTree tree
     - a query point query
     - the axis along which the root node of tree splits
     - the current nearest neighbor
    The function should return a tuple, which contains the distance
    to the query point and the location of the neighbor. For example the
    data points np.array([(1, 3), (1, 8), (2, 2), (2, 10), (3, 6), (4, 1), (5,
    4), (6, 8), (7, 4), (7, 7), (8, 2), (8, 5), (9, 9)]) should return for a
    query point [4,8] the result (2.0, array([6, 8])).
    """

    if not tree:
        return None, None

    k = tree.location.shape[0]

    if query[axis] <= tree.location[axis]:
        sub_tree = tree.left_child
        other_tree = tree.right_child
    else:
        sub_tree = tree.right_child
        other_tree = tree.left_child

    # Get the nearest neighbor from the subtree that our point is in.
    # (Downwards pass)
    distance, neighbor = one_NN_rec(sub_tree, query, axis=(axis + 1) % k)
    if neighbor is None:
        neighbor = tree.location
        distance = norm(neighbor - query)

    # Check if parent is closer than the NN from the subtree
    # (Upwards pass)
    if norm(tree.location - query) < distance:
        neighbor = tree.location
        distance = norm(neighbor - query)

    # Check if our query could have a neighrest neighbor (which better than our current candidate)
    # in the other subtree
    if other_tree:
        axis_dist = abs(query[axis] - tree.location[axis])
        if axis_dist <= distance:
            tmpDistance, tmpNeighbor = one_NN_rec(
                other_tree, query, axis=(axis + 1) % k
            )
            if tmpDistance < distance:
                distance = tmpDistance
                neighbor = tmpNeighbor

    return distance, neighbor

Now execute the code.

In [ ]:
tree = kdtree(
    np.array(
        [
            (1, 3),
            (1, 8),
            (2, 2),
            (2, 10),
            (3, 6),
            (4, 1),
            (5, 4),
            (6, 8),
            (7, 4),
            (7, 7),
            (8, 2),
            (8, 5),
            (9, 9),
        ]
    ),
    0,
)
n_neighbors = 2

In [ ]:
one_NN_rec(tree, [4, 8])

We see that the distance is 2 and the NN is $(6,8)$.

### b)

In [ ]:
def takeSecond(elem):
    return elem[1]

In [ ]:
def kNN_rec(tree, axis, query, neighbors, n_neighbors):
    """
    This recursive function accepts
     - a KDTree tree
     - the axis along which the root node of tree splits
     - a query point query
     - a current list of neighbors, sorted with ascending distance
       (list of pairs each containing the distance and the point itself)
     - the desired number of neighbors n_neighbors
    and modifies the neighbors list so that
     - points from the tree are added while we have fewer than n_neighbors
     - all closer points from the tree replace existing neighbors once weneighbor
       reached n_neighbors
    It returns the number of nodes that were visited during traversal and modifies
    the neighbors-object in-place.
    """

    if not tree:
        return 0

    k = tree.location.shape[0]

    if query[axis] <= tree.location[axis]:
        sub_tree = tree.left_child
        other_tree = tree.right_child
    else:
        sub_tree = tree.right_child
        other_tree = tree.left_child

    # Get the nearest neighbors from the subtree that our point is in.
    # (Downwards pass)
    travel_count = kNN_rec(
        sub_tree, (axis + 1) % k, query, neighbors, n_neighbors
    )

    # Add the root to the NN list and remove the most distant one
    neighbors.append(
        (tree.location, norm(tree.location - query)),
    )
    neighbors.sort(key=takeSecond)
    travel_count += 1

    if len(neighbors) > n_neighbors:
        neighbors.pop(-1)

    # Check the other subtree for any better candidates if there could be any
    if other_tree:
        axis_dist = abs(query[axis] - tree.location[axis])
        if axis_dist <= neighbors[-1][1]:
            return travel_count + kNN_rec(
                other_tree,
                (axis + 1) % k,
                query,
                neighbors,
                n_neighbors,
            )

    return travel_count

### c)

First define the brute force algorithm.

In [ ]:
def brute_force_knn(data, query, n_neighbors):
    neighbors_w_dist = []
    for point in data:
        neighbors_w_dist.append(
            (point, norm(point - query)),
        )
    neighbors_w_dist.sort(key=takeSecond)
    neighbors_w_dist = neighbors_w_dist[:n_neighbors]
    return neighbors_w_dist

In [ ]:
def init_random(amount, dim):
    return np.random.rand(amount, dim)


def get_rand_query(dim):
    return np.random.rand(dim)

In [ ]:
n_points = 10**5
dim = 5

data = init_random(n_points, dim)
tree = kdtree(data)

In [ ]:
data = init_random(n_points, dim)
tree = kdtree(data)
query = get_rand_query(dim)

neighbors = []
kNN_rec(tree, 0, query, neighbors, n_neighbors)

# See below
brute_force_neighbors = np.array(
    list(zip(*brute_force_knn(data, query, n_neighbors)))[0]
)

In [ ]:
# zip(*neighbors) will return a list with two elements
# The first element is a tuple of all datapoints, the second a tuple of all distances
neighbors = np.array(list(zip(*neighbors))[0])

# Check if all points are equal
np.equal(neighbors, brute_force_neighbors).all()

As we see from above the nearest neighbors from the brute force algorithm and our implementation are equal.

### d)

In [ ]:
sizes = [10, 100, 1000, 10000, 100000]
# Take 5 queries as in the sheet
queries = [get_rand_query(dim) for _ in range(5)]
# Since the data is random uniform anyway we dont really have to take a random subset
subsets = [data[:size] for size in sizes]
trees = [kdtree(subset) for subset in subsets]

for query in queries:
    visited_counts = []
    for tree in trees:
        neighbors = []

        visited_count = kNN_rec(tree, 0, query, neighbors, n_neighbors=10)
        visited_counts.append(visited_count)

    plt.plot(
        sizes,
        visited_counts,
        label=f"query: {query}",
    )

plt.xscale("log")
plt.xlabel("number of points")
plt.ylabel("nodes visited")
plt.legend()

### e)

We assume that the task was misspelled and we should repeat the experiment from d).

From our plots we see that with a higher dimension we visit more nodes.

In [ ]:
n_points = 10**5

for dim in [1, 3, 5, 10, 20, 30]:
    plt.figure(dim)
    data = init_random(n_points, dim)

    sizes = [10, 100, 1000, 10000, 100000]
    # Take 5 queries as in the sheet
    queries = [get_rand_query(dim) for _ in range(5)]
    # Since the data is random uniform anyway we dont really have to take a random subset
    subsets = [data[:size] for size in sizes]
    trees = [kdtree(subset) for subset in subsets]

    for query in queries:
        visited_counts = []
        for tree in trees:
            neighbors = []

            visited_count = kNN_rec(tree, 0, query, neighbors, n_neighbors=10)
            visited_counts.append(visited_count)

        plt.plot(
            sizes,
            visited_counts,
            label=f"query: {query}",
        )

    plt.xscale("log")
    plt.xlabel("number of points")
    plt.ylabel("nodes visited")
    plt.legend()
    plt.title(f"dimension: {dim}")